In [ ]:
!pip install torch
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 51.2 MB/s eta 0:00:00


In [ ]:
import os
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import seed_everything

In [ ]:
class ConvModule(nn.Module):
    def __init__(self, in_dim, out_dim, modes):
        super(ConvModule, self).__init__()
        self.ind = in_dim
        self.outd = out_dim
        self.modes = modes

        self.weights = nn.Parameter(torch.rand(self.ind, self.outd, self.modes, dtype=torch.cfloat))

    def product(self, input, weights):
        #returns multiplication of input (batch, ind, d) and weights (ind, outd, d)
        #final dimensions are (batch, outd, d)

        return torch.einsum("bid,iod->bod", input, weights)

    def forward(self, x):
        #fourier transform and inverse feed-forward
        x_ft = torch.fft.rfft(x)

        out_ft = torch.zeros(x.shape[0], self.outd, x.shape[-1]//2+1, device = x.device, dtype = torch.cfloat)

        out_ft[:, :, :self.modes] = self.product(x_ft[:,:,:self.modes], self.weights)

        out_x = torch.fft.irfft(out_ft)
        return out_x

In [ ]:
class FNOModule1D(nn.Module):
    def __init__(self, modes, lifted_dim):
        super(FNOModule1D, self).__init__()
        self.modes = modes
        self.lifted_dim = lifted_dim

        self.liftLayer = nn.Linear(3, lifted_dim)

        self.fLayer1 = ConvModule(self.lifted_dim, self.lifted_dim, self.modes)
        self.fLayer2 = ConvModule(self.lifted_dim, self.lifted_dim, self.modes)
        self.fLayer3 = ConvModule(self.lifted_dim, self.lifted_dim, self.modes)
        self.fLayer4 = ConvModule(self.lifted_dim, self.lifted_dim, self.modes)

        self.w1 = nn.Conv1d(self.lifted_dim, self.lifted_dim, 1)
        self.w2 = nn.Conv1d(self.lifted_dim, self.lifted_dim, 1)
        self.w3 = nn.Conv1d(self.lifted_dim, self.lifted_dim, 1)
        self.w4 = nn.Conv1d(self.lifted_dim, self.lifted_dim, 1)

        self.fc = nn.Linear(self.lifted_dim, 128)
        self.fcOut = nn.Linear(128, 1)

    def forward(self, x):
        x = self.liftLayer(x) #dimension converted (batch, d, 3) ---> (batch, d, lifted)
        x = x.permute(0,2,1)

        x1 = self.fLayer1(x)
        b1 = self.w1(x)
        x = x1 + b1
        x = F.relu(x)

        x1 = self.fLayer2(x)
        b1 = self.w2(x)
        x = x1 + b1
        x = F.relu(x)

        x1 = self.fLayer3(x)
        b1 = self.w3(x)
        x = x1 + b1
        x = F.relu(x)

        x1 = self.fLayer4(x)
        b1 = self.w4(x)
        x = x1 + b1
        #x = F.relu(x)
        x = x.permute(0, 2, 1)
        x = self.fc(x)
        x = F.relu(x)
        x = self.fcOut(x)

        return x

#Training and Testing

Currently looking for a 2D Darcy Flow dataset